In [4]:
import requests
import json
import pandas as pd
import numpy as np

headers = {
  "Client-Id": '740209',
  "Api-Key": '73615d31-68e7-4c00-a008-e53b9d08d600'
}

{'categoryId': 666, 'sku': 228}


In [4]:



def send_product(sku_arr: list[list[int]]):
    response = requests.post('https://api-seller.ozon.ru/v1/product/import-by-sku', json={
        "items": [
            {
                "sku": sku,
                "name": str(sku),
                "offer_id": str(sku),
                "currency_code": "RUB",
                "old_price": "2590",
                "price": "2300",
                "premium_price": "2200",
                "vat": "0.1"
            }
            for sku in sku_arr
        ]
    }, headers=headers)
    print(response)
    result = response.json()
    print(result)


df = pd.read_csv('dataset.csv')
df.head()
print(len(df))
df = df.drop_duplicates('categoryId')
print(len(df))
df = df['sku']
for sku_arr in np.array_split(df, len(df) // 500 + 1):
    send_product(sku_arr)


20749
5907


c:\Python310\lib\site-packages\numpy\lib\shape_base.py:790: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  sub_arys.append(_nx.swapaxes(sary[st:end], axis, 0))


<Response [200]>
{'result': {'task_id': 609858620, 'unmatched_sku_list': []}}


KeyboardInterrupt: 

In [9]:
# Read raw json and puts in df
def prepare_json(path: str):
    obj = json.loads(open(path, 'r', encoding='utf-8').read())
    data = []
    for key in obj:
        row = obj[key]
        if row['title'] is not None and row['description'] is not None and row['category'] is not None and row['hierarchy'] is not None:
            if row['title'].strip() != '' and row['description'].strip() != '' and row['category'].strip() != '' and row['hierarchy'].strip() != '':
                tree = row['hierarchy'].split('/')
                row['category'] = str([tree[min(5, len(tree) - 1)]])
                # row['category'] = str([row['category']])
                row['category'] = str(tree[:3])

                data.append(obj[key])

    df = pd.json_normalize(data)
    df = df.rename(columns={"title": "titles",
                   "category": "terms", 'description': 'summaries'})
    df = df[[
        'titles', 'summaries', 'terms', 'sku', 'categoryId', 'url'
    ]]
    return df

# Fixes json to final dataset


def fix_categories(dataset: str, out: str):
    response = requests.post('https://api-seller.ozon.ru/v3/products/info/attributes', json={
        "filter": {

            "visibility": "ALL"
        },
        "limit": 1000,
        "sort_dir": "ASC"
    }, headers=headers)
    print(response)
    result = [
        {
            'trueCategoryId': str(old_dict['category_id']),
            'sku': int(old_dict['name'])
        }
        for old_dict in response.json()['result'] if old_dict['name'].isdecimal()
    ]
    print(len(result))
    df = prepare_json(dataset)
    print(len(df))
    # df = df.drop_duplicates('categoryId')
    df = pd.merge(df,
                  pd.json_normalize(result), on='sku',
                  how='left')

    # inject categories tree
    tree = requests.post(
        'https://api-seller.ozon.ru/v2/category/tree', headers=headers)
    print(tree)
    tree = tree.json()['result']

    tree_map = []
    stack = []
    def dfs(dict: dict | list,
            stack: list[any],
            start=False):
        children = dict
        if not start:
            children = dict['children']
            
            # full
            # stack.append({
            #     'category_id': dict['category_id'],
            #     'title': dict['title']
            # })
            # titles only
            stack.append(
                dict['title']
            )
            
            
        if len(children) == 0:
            tree_map.append({'trueCategoryId': str(dict['category_id']), 'trueHierarchy': stack.copy()})
        else:
            for item in children:
                dfs(item, stack)
        if not start: stack.pop()
    dfs(tree, stack, True)
    
    df = pd.merge(df,
                pd.json_normalize(tree_map), on='trueCategoryId',
                how='left')
    
    json.dump(tree, open('./parse/categories_tree.json', 'w+', encoding='utf-8'),
              indent=2, ensure_ascii=False)
    df.to_csv(out)
    json.dump(tree_map,
              open('./sus.json', 'w+', encoding='utf-8'),
              indent=2, ensure_ascii=False)
    return df


df = fix_categories(
    './parse/products_data/found.json',
    './dataset.csv')
print(len(df))
df.head()


<Response [200]>
999
20749
<Response [200]>
20749


,titles,summaries,terms,sku,categoryId,url,trueCategoryId,trueHierarchy
0,Нож для яблок Яблокорезка Прибор для удаления ...,Удалитель у яблок предназначен для аккуратного...,"['Дом и сад', 'Посуда', 'Кухонные принадлежнос...",701414369,14517,https://www.ozon.ru/product/nozh-dlya-yablok-y...,50765808,"[Дом и сад, Ножи кухонные, Удалитель сердцевин..."
1,"Набор доктора игровой 2550 ""Волшебная аптечка""...",• В наборе медицинские инструменты (25 предмет...,"['Детские товары', 'Игрушки и игры', 'Сюжетно-...",178962428,7112,https://www.ozon.ru/product/nabor-doktora-igro...,17039241,"[Детские товары, Игрушки, Сюжетно-ролевой набо..."
2,Икона Святитель Николай Чудотворец,"Святитель Николай Мирликийский, или как его ещ...","['Дом и сад', 'Авторские товары ручной работы'...",536003504,35346,https://www.ozon.ru/product/ikona-svyatitel-ni...,17036878,"[Дом и сад, Авторские товары, Иконы авторские]"
3,"Сухое молоко цельное 1000г, 1 кг",сухое молоко цельное 1 кг,"['Продукты питания', 'Масла, соусы, специи', '...",265143538,9410,https://www.ozon.ru/product/suhoe-moloko-tseln...,101031443,"[Продукты питания, Соль, сахар, специи, Сухое ..."
4,"Мука ржаная цельнозерновая из био зерна, рожь,...",Рожь для этой муки была выращена без использов...,"['Продукты питания', 'Макароны, крупы, мука', ...",257993749,1620,https://www.ozon.ru/product/muka-rzhanaya-tsel...,17032557,"[Продукты питания, Макаронные изделия, крупы, ..."
